In [1]:
import pandas as pd

# Compile results from multiple runs

In [2]:
PARENT_DIR = '../variance_model/results/64u_test_run#'
COMPILED_RESULTS_FILE_NAME = '64u_compiled_results.csv'

In [3]:
fastcharge_basemodel_result_file = '/fastcharge_basemodel_result_RUN#'
normalcharge_basemodel_result_file = '/normalcharge_basemodel_result_RUN#'
normalcharge_transfermodel_result_file = '/normalcharge_transfermodel_result_RUN#'
NUM_TEST_PERFORMED = 5
compiled_data = { 'Test No.': [], 'models': [], 'RMSE': [], 'MAE': [], 'MSE': [] }

def append_results(df, num):
    compiled_data['models'].append(df['models'].values[0])
    compiled_data['RMSE'].append(df['RMSE'].values[0])
    compiled_data['MAE'].append(df['MAE'].values[0])
    compiled_data['MSE'].append(df['MSE'].values[0])
    compiled_data['Test No.'].append(num)

def compile_data():
    # read all the files and store RMSE value of the models into a dictionary
    for i in range(1, NUM_TEST_PERFORMED + 1):
        fc_basemodel_res_fp = PARENT_DIR + str(i)  + fastcharge_basemodel_result_file + str(i) + '.csv'
        nc_basemodel_res_fp = PARENT_DIR + str(i)  + normalcharge_basemodel_result_file + str(i) + '.csv'
        nc_transfermodel_res_fp = PARENT_DIR + str(i)  + normalcharge_transfermodel_result_file + str(i) + '.csv'

        fc_basemodel_res = pd.read_csv(fc_basemodel_res_fp)
        nc_basemodel_res = pd.read_csv(nc_basemodel_res_fp)
        nc_transfermodel_res = pd.read_csv(nc_transfermodel_res_fp)

        fc_basemodel_res.rename(columns={'Unnamed: 0': 'models'}, inplace=True)
        nc_basemodel_res.rename(columns={'Unnamed: 0': 'models'}, inplace=True)
        nc_transfermodel_res.rename(columns={'Unnamed: 0': 'models'}, inplace=True)

        append_results(fc_basemodel_res, i)
        append_results(nc_basemodel_res, i)
        append_results(nc_transfermodel_res, i)

    # convert the dictionary to dataframe
    compiled_data = pd.DataFrame(compiled_data)

    #sort by models
    compiled_data = compiled_data.sort_values(by=['models'])

    #average the results for variance_fastcharge_basemodel
    compiled_data['RMSE_avg'] = compiled_data['RMSE'].groupby(compiled_data['models']).transform('mean')
    compiled_data['MAE_avg'] = compiled_data['MAE'].groupby(compiled_data['models']).transform('mean')
    compiled_data['MSE_avg'] = compiled_data['MSE'].groupby(compiled_data['models']).transform('mean')

    # save to csv file without the index
    compiled_data.to_csv('../variance_model/results/' + COMPILED_RESULTS_FILE_NAME, index=False)    

In [70]:
def collate_data():
    node64 = pd.read_csv('../variance_model/results/64u_compiled_results.csv')
    node16 = pd.read_csv('../variance_model/results/16u_compiled_results.csv')
    node1 = pd.read_csv('../variance_model/results/compiled_results.csv')

    node64.iloc[[0,5,10],[1]] = node64.iloc[[0,5,10],[1]] + '_64u'
    node16.iloc[[0,5,10],[1]] = node16.iloc[[0,5,10],[1]] + '_16u'
    node1.iloc[[0,5,10],[1]] = node1.iloc[[0,5,10],[1]] + '_1u'

    x= node64.iloc[[0, 5, 10], [1,5,6,7]]
    y= node16.iloc[[0, 5, 10], [1,5,6,7]]
    z= node1.iloc[[0, 5, 10], [1,5,6,7]]

    res = pd.concat([x, y, z], axis=0)
    res.sort_index(inplace=True)
    res.to_csv('../variance_model/results/collated_results.csv', index=False)


In [72]:
def main():
    # compile_data()
    collate_data()

main()